# <h1 style="font-size:50px;">INTRODUCTION</h1> 


<p style="font-size:18px">This notebook contains of Turkey's current Covid-19 datas. Prepared to be useful to you!</p>



## <nav style="font-size:30px">Contents</nav>

1. [<b style="font-size:18px">Overview of Data</b>](#1)
2. [<b style="font-size:18px">Correlations</b>](#2)
3. [<b style="font-size:18px">Total Cases & Deaths by Months</b>](#3)
4. [<b style="font-size:18px">Total Cases by Regions</b>](#4)
5. [<b style="font-size:18px">Peak Day</b>](#5)
6. [<b style="font-size:18px">Distribution Ratios Of Total Cases by Months</b>](#6)
7. [<b style="font-size:18px">Numbers of Cases & Recoverings in Months</b>](#7)
8. [<b style="font-size:18px">Outlier Analysis</b>](#8)
9. [<b style="font-size:18px">Number of Total Tests by Months and Positive Test Rate Means</b>](#9)
10. [<b style="font-size:18px">Intubated and Intensive Cares</b>](#10)
11. [<b style="font-size:18px">Numbers of Seriously Ill and Pneumonia Ratios</b>](#11)

<a id="1"></a>
# Overview of Data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import plotly.graph_objs as go
import plotly as py
from plotly.offline import iplot
import missingno as msno

In [ ]:
dataset = pd.read_csv("/kaggle/input/turkeycovid19basic/TurkeyCovid19Dataset.csv")
seriouslyill= pd.read_csv("/kaggle/input/turkeycovid19basic/SeriouslyIll.csv")
regiondist = pd.read_csv("/kaggle/input/turkeycovid19basic/DistributionByRegions.csv")

In [ ]:
dataset.info()

In [ ]:
for i in dataset.columns:
    try: assert dataset[i].notnull().all()
    except AssertionError: 
        numbernan = dataset[i].isnull().sum()
        print("There is {} NaN values in {} column!".format(numbernan,i))

In [ ]:

nanfig,nanax=plt.subplots(tight_layout=True,figsize=(10,5))
msno.bar(dataset,color="tomato",fontsize=13,ax=nanax)
nanax.grid(True,alpha=0.4,linestyle="--",linewidth=1)
nanax.set_yticks(np.arange(0,1.1,0.1))
nanax.set_title("Number Of Not NaN Values In The Columns",fontsize=20)
nanax.set_ylabel("%",fontsize=15)
plt.show()

In [ ]:
dataset.head()

In [ ]:
dataset.describe()

In [ ]:
#Total tests till now
print("Total Tests: {}".format(dataset["Daily Test"].sum())) 

In [ ]:
#Total cases till now
print("Total Cases: {}".format(dataset["Daily Cases"].sum()))

In [ ]:
#Total deaths till now
print("Total Deaths: {}".format(dataset["Daily Deaths"].sum()))

In [ ]:
#Total recovering people till now
print("Total Recovering: {}".format(dataset["Daily Recovering"].sum()))

In [ ]:
# We will use this method for added months column on dataset
def months(n):
    mlist = n.split()
    month = mlist[1]
    return month

In [ ]:
type(dataset["Date"].apply(months))

In [ ]:
# Added new column with insert() and apply() methods 
dataset.insert(0,"Months",dataset["Date"].apply(months))

In [ ]:
# Lets see this
dataset.head()

In [ ]:
# Added new column about positive case ratio
dataset["positiveRatio"] = dataset["Daily Cases"]/dataset["Daily Test"]*100

In [ ]:
# Control new column
dataset.head()

<a id="2"></a>
# Correlations

In [ ]:
# Correlation table for this data

dataset.corr()

In [ ]:
heatfig,heatax = plt.subplots(figsize=(10,8),tight_layout=True)
sns.heatmap(dataset.corr(),annot=True)
heatax.set_title("Correlation Map",fontsize=20)
heatax.tick_params(width=3,color="grey",labelsize=11)
plt.show()

In [ ]:
# Grouped by months
datasetGroup = dataset.groupby("Months")

In [ ]:
# lets take one sample
datasetGroup.head(1)

In [ ]:
totalDataCorr = datasetGroup.sum().copy()
totalDataCorr = totalDataCorr.rename(columns={"Daily Test":"Total Test by months","Daily Cases":"Total Cases by months","Daily Deaths":"Total Deaths by months","Daily Recovering":"Total Recovering by months"})
totalDataCorr.drop(["Total Intensive Care","Total Intubated","positiveRatio"],axis=1,inplace=True)

In [ ]:
# Correlation table of aggregate datas by months
totalDataCorr.corr()

In [ ]:
corrfig,corrax = plt.subplots(figsize=(8,6),tight_layout=True)
sns.heatmap(totalDataCorr.corr(),annot=True)
corrax.set_title("Correlation Map of Aggregate Datas by Months",fontsize=15,color="blue")
corrax.tick_params(width=3,color="grey")
plt.show()

### If we look heatmap above, we can say total cases and total deaths are linearly proportional.

<a id="3" ></a>
# Total Cases & Deaths by Months

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(12,9),tight_layout=True)

sns.barplot(x = datasetGroup.sum().index,y = datasetGroup.sum()["Daily Cases"].values,palette="rocket",ax=ax1)
ax1.set_yticks(np.arange(0,115000,5000))
ax1.grid(alpha=0.25,color="grey")
ax1.set_ylabel("Total Cases",fontsize = 15)
ax1.set_xlabel("Months",fontsize = 15)
ax1.set_title("Total Cases by Months",fontsize=20)
ax1.tick_params(width = 5,color="grey",grid_linewidth=1,grid_linestyle="--",labelsize=10)


sns.barplot(x=datasetGroup.sum().index,y= datasetGroup.sum()["Daily Deaths"].values,palette="vlag",ax=ax2)
ax2.grid(alpha=0.25,color="grey")
ax2.set_yticks(np.arange(0,3150,150))
ax2.set_ylabel("Total Deaths",fontsize=15)
ax2.set_xlabel("Months",fontsize=15)
ax2.set_title("Total Deaths by Months",fontsize=20)
ax2.tick_params(width=5,color="grey",grid_linestyle="--",grid_linewidth=1,labelsize=10)


fig.autofmt_xdate(rotation=90,ha="center")
plt.show()


#### April is the month with the most cases and deaths

In [ ]:
# The number of total cases and total deaths by months

for i,j,k in zip(datasetGroup.sum()["Daily Cases"].index,datasetGroup.sum()["Daily Cases"].values,datasetGroup.sum()["Daily Deaths"].values):
    print("{}:\nTotal Cases: {} \t Total Deaths: {}\n".format(i,j,k))

<a id="4"></a> 
# Total Cases by Regions

In [ ]:
regvalues = regiondist.iloc[0].drop("WEEK")
regvalues

In [ ]:
tracereg = go.Scatter(
        x=regvalues.index,
        y=regvalues.values,
        mode="markers",
        marker=dict(color=regvalues.values,size=list(regvalues.values/1200),showscale=True,colorscale="thermal",line=dict(width=2,color="#111")),
        text = regvalues.index,
)
tracelay = dict(title="Total Cases by Regions",font=dict(size=15))
figreg = go.Figure(data=tracereg,layout=tracelay)
figreg.update_layout(
    autosize=True,
    paper_bgcolor="lightblue",
    plot_bgcolor="lightblue",
    modebar=dict(activecolor="#111"),
)
iplot(figreg)

<a id="5"></a>
# Peak Day

<font style="font-size:16px; letter-spacing:0.5px;">Numbers of cases began increase again. The old peak day was 11 April. </font>

In [ ]:
# The peak day

peakday = dataset[dataset["Daily Cases"] == dataset.drop(0,axis=0)["Daily Cases"].nlargest(1).iloc[0]]
peakday

In [ ]:
print("Peak day is {}\nnumber of cases: {}".format(peakday.Date.iloc[0],peakday["Daily Cases"].iloc[0]))

In [ ]:
peakfig,(peakax1,peakax2) = plt.subplots(nrows=1,ncols=2,figsize=(12,6),tight_layout=True)
peakax1.plot(dataset.Date.drop(0,axis=0),dataset["Daily Cases"].drop(0,axis=0),color="black",lw=3,marker = "o",markerfacecolor="orange",label="Cases by days")
peakax1.set_xlim("1 April","20 April")
peakax1.grid(True,alpha=0.4,color="grey")
peakax1.tick_params(grid_linewidth=1,width=5,color="orange",grid_linestyle="--")
peakax1.set_xlabel("April",fontsize=15)
peakax1.set_ylabel("Number of Cases",fontsize=15)
peakax1.set_title("Old Peak Day",fontsize=20)
peakax1.set_xticklabels(dataset.Date.drop(0,axis=0),rotation=90,ha="center")
peakax1.legend(loc = "center right",fontsize=10)


peakax2.plot(dataset.Date.drop(0,axis=0),dataset["Daily Cases"].drop(0,axis=0),color="black",lw=3,marker = "o",markerfacecolor="orange",label="Cases by days")
peakax2.set_xlim("10 November",dataset.drop(0,axis=0).set_index("Date")["Daily Cases"].nlargest(1).index[0])
peakax2.grid(True,alpha=0.4,color="grey")
peakax2.tick_params(grid_linewidth=1,width=5,color="orange",grid_linestyle="--")
peakax2.set_xlabel("November",fontsize=15)
peakax2.set_ylabel("Number of Cases",fontsize=15)
peakax2.set_title("Current Peak Day",fontsize=20)
peakax2.set_xticklabels(dataset.Date.drop(0,axis=0),rotation=90,ha="center")
peakax2.set_yticks(np.arange(1000,6000,500))
peakax2.legend(loc = "center right",fontsize=10)



plt.show()

<a id="6"></a>
# Distribution Ratios Of Total Cases by Months

In [ ]:
pie = dict(
    values = datasetGroup.sum()["Daily Cases"].values,
    labels= datasetGroup.sum().index,
    domain= dict(x= [0,0.5]),
    name=" Cases",
    hoverinfo="value+name",
    hole=0.3,
    type="pie",
    textinfo="label+percent",
    marker=dict(line=dict(width=2,color="#111")),
    pull=[0.0]+[0]*(len(datasetGroup.sum().index)-1)
)
layout = dict(title="Distribution Ratios Of Total Cases by Months")
figplot = go.Figure(data=pie,layout=layout)
figplot.update_layout(
    autosize=True,
    title=dict(x=0.05),
    legend=dict(bordercolor="#444",borderwidth=1,orientation="v",x=0.75),
    paper_bgcolor="#d8d8d8",
    modebar=dict(activecolor="#111"),
    hoverlabel=dict(bordercolor="#111")
)
iplot(figplot)


In [ ]:
colors=['gold', 'mediumturquoise', 'darkorange', 'lightgreen',"tomato","lightblue","cyan","green","red"]
pie2 = dict(
    values = datasetGroup.sum()["Daily Deaths"].values,
    labels= datasetGroup.sum().index,
    domain= dict(x= [0,0.5]),
    name=" Deaths",
    hoverinfo="value+name", # text = datasetGroup.sum()["Daily Deaths"].values
    hole=0.3,
    type="pie",
    textinfo="label+percent",
    marker = dict(colors= colors,line=dict(width=2,color="#111")),
)
layout2 = dict(title="Distribution Ratios Of Total Deaths by Months")
figplot2 = go.Figure(data=pie2,layout=layout2)
figplot2.update_layout(
    autosize=True,
    title=dict(x=0.05),
    legend=dict(borderwidth=1,bordercolor="#444",x=0.75),
    paper_bgcolor="#d8d8d8",
    modebar=dict(activecolor="#111"),
    hoverlabel=dict(bordercolor="#111")
    
    
)
iplot(figplot2)

In [ ]:
mlist = dataset.Months.unique()
for i in mlist:
    ratioc = datasetGroup.sum()["Daily Cases"][i] / dataset["Daily Cases"].sum() * 100
    ratiod = datasetGroup.sum()["Daily Deaths"][i] / dataset["Daily Deaths"].sum() * 100
    print("{}:\nDistribution ratio of cases: %{:.1f}\nDistribution ratio of deaths: %{:.1f}\n".format(i,ratioc,ratiod))

<a id="7"></a>
# Numbers of Cases & Recoverings in Months

### Slicing dataset with two features by months

In [ ]:
marchCase = dataset.drop(0,axis=0).set_index("Date").loc["28 March":"31 March",["Daily Cases","Daily Recovering"]]
aprilCase = dataset.set_index("Date").loc["1 April":"30 April",["Daily Cases","Daily Recovering"]]
mayCase = dataset.set_index("Date").loc["1 May":"31 May",["Daily Cases","Daily Recovering"]]
juneCase = dataset.set_index("Date").loc["1 June":"30 June",["Daily Cases","Daily Recovering"]]
julyCase = dataset.set_index("Date").loc["1 July":"31 July",["Daily Cases","Daily Recovering"]]
augustCase = dataset.set_index("Date").loc["1 August":"31 August",["Daily Cases","Daily Recovering"]]
septCase= dataset.set_index("Date").loc["1 September":"30 September",["Daily Cases","Daily Recovering"]]
octCase= dataset.set_index("Date").loc["1 October":"31 October",["Daily Cases","Daily Recovering"]]
novCase = datasetGroup.get_group("November").loc[:,["Date","Daily Cases","Daily Recovering"]]

In [ ]:
figcase,(axcase1,axcase2) = plt.subplots(1,2,figsize=(12,9),tight_layout=True)
sns.barplot(x = marchCase["Daily Cases"].values,y = marchCase.index,ax=axcase1,color="red",label="Daily Cases")
sns.barplot(x = marchCase["Daily Recovering"].values,y = marchCase.index,ax=axcase1,color="green",label = "Daily Recoverings")
axcase1.grid(color="grey",alpha=0.3)
axcase1.tick_params(width=3,color="black",grid_linewidth = 1, grid_linestyle = "--")
axcase1.set_ylabel("Dates",fontsize=15)
axcase1.set_xlabel("Number of Cases & Recoverings",fontsize=15)
axcase1.set_title("March",fontsize=20)
axcase1.set_xticks(np.arange(0,3000,250))
axcase1.legend(loc = "upper right",frameon=True)
sns.barplot(x = aprilCase["Daily Cases"],y = aprilCase.index,ax=axcase2,color="red",label="Daily Cases")
sns.barplot(x = aprilCase["Daily Recovering"],y = aprilCase.index,ax=axcase2,color="green",label = "Daily Recoverings",alpha=0.9)
axcase2.legend(loc = "upper right",frameon=True)
axcase2.grid(color="grey",alpha=0.3)
axcase2.tick_params(width=3,color="black",grid_linewidth=1,grid_linestyle="--")
axcase2.set_xlabel("Number of Cases & Recoverings",fontsize=15)
axcase2.set_ylabel("Dates",fontsize=15)
axcase2.set_title("April",fontsize=20)
plt.show()

In [ ]:
figcase2,(axcase3,axcase4) = plt.subplots(1,2,figsize=(12,9),tight_layout=True)
sns.barplot(x = mayCase["Daily Recovering"].values,y= mayCase.index,color="green",label="Daily Recoverings",ax=axcase3)
sns.barplot(x = mayCase["Daily Cases"].values,y = mayCase.index,color = "red",label = "Daily Cases",ax=axcase3,alpha=0.9)
axcase3.legend(loc="lower right",frameon=True)
axcase3.grid(color="grey",alpha=0.3)
axcase3.tick_params(width=3,color="black",grid_linewidth=1,grid_linestyle="--")
axcase3.set_xlabel("Number of Cases & Recoverings",fontsize=15)
axcase3.set_ylabel("Dates",fontsize=15)
axcase3.set_title("May",fontsize=20)
axcase3.set_xticks(np.arange(0,6000,500))
sns.barplot(x = juneCase["Daily Recovering"],y=juneCase.index,color="green",label="Daily Recoverings",ax=axcase4)
sns.barplot(x = juneCase["Daily Cases"],y = juneCase.index,color="red",label="Daily Cases",ax=axcase4,alpha=0.9)
axcase4.grid(color="grey",alpha=0.3)
axcase4.tick_params(width=3,color="black",grid_linewidth=1,grid_linestyle="--")
axcase4.legend(loc="lower right",frameon=True)
axcase4.set_xlabel("Number of Cases & Recoverings",fontsize=15)
axcase4.set_ylabel("Dates",fontsize=15)
axcase4.set_title("June",fontsize=20)
plt.show()

In [ ]:
figcase3,(axcase5,axcase6) = plt.subplots(1,2,figsize=(12,9),tight_layout = True)
sns.barplot(x = julyCase["Daily Recovering"],y = julyCase.index,ax=axcase5,color="green",label="Daily Recoverings")
sns.barplot(x = julyCase["Daily Cases"],y = julyCase.index,ax=axcase5,color="red",label="Daily Cases")
axcase5.grid(color="grey",alpha=0.3)
axcase5.tick_params(width=3,color="black",grid_linestyle="--",grid_linewidth=1)
axcase5.legend(loc="lower right",frameon=True)
axcase5.set_xlabel("Number of Cases & Recoverings",fontsize=15)
axcase5.set_ylabel("Dates",fontsize=15)
axcase5.set_title("July",fontsize=20)
sns.barplot(x=augustCase["Daily Cases"],y=augustCase.index,color="red",ax=axcase6,label="Daily Cases")
sns.barplot(x=augustCase["Daily Recovering"],y=augustCase.index,color="green",ax=axcase6,label="Daily Recoverings",alpha=0.9)
axcase6.grid(color="grey",alpha=0.3)
axcase6.tick_params(width=3,color="black",grid_linewidth=1,grid_linestyle="--")
axcase6.set_title("August",fontsize=20)
axcase6.set_xlabel("Number of Cases & Recoverings",fontsize=15)
axcase6.set_ylabel("Dates",fontsize=15)
axcase6.legend(loc="lower right",frameon=True)
plt.show()


In [ ]:
figcase4,(axcase7,axcase8) = plt.subplots(nrows=1,ncols=2,figsize=(12,9),tight_layout=True)
sns.barplot(x=septCase["Daily Cases"].values,y=septCase.index,ax=axcase7,color="red",label="Daily Cases")
sns.barplot(x=septCase["Daily Recovering"].values,y=septCase.index,ax=axcase7,color="green",alpha=0.9,label="Daily Recoverings")
axcase7.grid(True,color="gray",linestyle="--",linewidth=1,alpha=0.3)
axcase7.tick_params(width=3,color="black")
axcase7.legend(frameon=True,loc="lower right")
axcase7.set_xlabel("Number of Cases & Recoverings",fontsize=15)
axcase7.set_ylabel("Dates",fontsize=15)
axcase7.set_title("September",fontsize=20)

sns.barplot(x=octCase["Daily Cases"].values,y=octCase.index,ax=axcase8,color="red",label="Daily Cases")
sns.barplot(x=octCase["Daily Recovering"].values,y=octCase.index,ax=axcase8,color="green",alpha=0.9,label="Daily Recoverings")
axcase8.grid(True,color="gray",linestyle="--",linewidth=1,alpha=0.3)
axcase8.tick_params(width=3,color="black")
axcase8.legend(frameon=True,loc="lower right")
axcase8.set_xlabel("Number of Cases & Recoverings",fontsize=15)
axcase8.set_ylabel("Dates",fontsize=15)
axcase8.set_title("October",fontsize=20)

plt.show()

In [ ]:
figcase5,axcase9=plt.subplots(nrows=1,ncols=1,figsize=(12,4.5),tight_layout=True)
sns.barplot(x=novCase.Date.values,y=novCase["Daily Cases"].values,color="red",label="Daily Cases",ax=axcase9)
sns.barplot(x=novCase.Date.values,y=novCase["Daily Recovering"].values,color="green",label="Daily Recoverings",ax=axcase9,alpha=0.9)
axcase9.set_yticks(np.arange(0,6500,500))
axcase9.grid(True,alpha=0.3,linestyle="--",linewidth=1,color="gray")
axcase9.tick_params(width=3,color="black")
axcase9.legend(loc="lower right",frameon=True)
axcase9.set_title("November",fontsize=20)
axcase9.set_xlabel("Dates",fontsize=15)
axcase9.set_ylabel("Number of Cases & Recoverings",fontsize=15)
plt.xticks(rotation=90)
plt.show()



<p style="font-size:18px;font-family:Arial;font-weight:bold;">As you can see above the increase of the daily cases started again, this is second wave of the pandemic. </p>

<a id="8"></a>
# Outlier Analysis

In [ ]:
figbox,axbox = plt.subplots(figsize=(10,6),tight_layout=True)
sns.boxplot(x = dataset.drop(0,axis=0)["Daily Cases"].values,y=dataset.drop(0,axis=0).Months,ax=axbox)
axbox.grid(color="grey",alpha=0.4,linestyle="--",linewidth=1)
axbox.set_xticks(np.arange(0,6000,500))
axbox.set_title("Outlier Analysis",fontsize=20)
axbox.set_ylabel("Number of Cases",fontsize=15)
axbox.set_xlabel("Months",fontsize=15)
axbox.tick_params(width=3,color="black")
plt.show()

Outlier data as you can see above is **very less**. The month with the highest change interval for the number of cases is **April**.

In [ ]:
figsw,axsw = plt.subplots(figsize=(10,6),tight_layout=True)
sns.swarmplot(y = dataset.drop(0,axis=0).Months,x = dataset.drop(0,axis=0)["Daily Cases"].values,ax=axsw,size=6)
axsw.grid(color="grey",alpha=0.4,linestyle="--",linewidth=1)
axsw.set_xticks(np.arange(0,6000,500))
axsw.set_ylabel("Number of Daily Cases",fontsize=15)
axsw.set_xlabel("Months",fontsize=15)
axsw.tick_params(width=3,color="black")
plt.show()

<a id="9"></a>
# Number of Total Tests by Months and Positive Test Rate Means

In [ ]:
print("Positive Test Rate Means\n")
ratios=list()
for i,v in zip(datasetGroup.mean()["positiveRatio"].index,datasetGroup.mean()["positiveRatio"].values):
    print("{}:\tratio: %{:.2f}\n".format(i,v))
    ratios.append((i,v))

In [ ]:
month_list = list()
for k in sorted(datasetGroup.mean().positiveRatio.values,reverse=True):
    for i,j in ratios:
        if j == k: 
            month_list.append(i)
            ratios.remove((i,j))
    

In [ ]:
figpos,axpos=plt.subplots(figsize=(12,6),tight_layout=True)
sns.barplot(x = month_list,y =sorted(datasetGroup.mean().positiveRatio.values,reverse=True) )
axpos.grid(color="grey",alpha=0.3,linestyle="--",linewidth=1)
axpos.set_title("Positive Test Rate Means",fontsize=20)
axpos.set_xlabel("Months",fontsize=15)
axpos.set_ylabel("Rates",fontsize=15)
axpos.tick_params(width=3,color="black")
axpos.set_yticks(np.arange(0,18,1))
plt.show()

In [ ]:
totalTest = datasetGroup.sum()["Daily Test"].sort_values()
for i,v in zip(totalTest.index,totalTest.values):
    t="\t"
    if i in ["November","September","October"]: t=" "
    print("{}:{} Total Test: {}\n".format(i,t,v))

In [ ]:
figtest,axtest = plt.subplots(figsize=(12,6),tight_layout=True)
sns.barplot(x = totalTest.index,y=totalTest.values,palette="rocket")
axtest.grid(color="grey",alpha=0.3,linestyle="--",linewidth=1)
axtest.set_ylabel("Total Test Done\n(10^6)",fontsize=15)
axtest.set_xlabel("Months",fontsize=15)
axtest.set_title("Number of Total Tests by Months",fontsize=20)
axtest.tick_params(width=3,color="black")
plt.show()

<a id="10"></a>
# Intubated and Intensive Cares

<pre style="font-size:16px;">
The number of intubated patients and intensive care patients was shared by the ministry until 28 July, after which these data were not shared. 
Therefore, these data could be collected until 28 July. After July 28, data on seriously ill and pneumonia were shared by the ministry. </pre>

In [ ]:
dropnaDataset = dataset.dropna(how="any",axis=0)
dropnaDataset.drop(0,axis=0,inplace=True)

In [ ]:
# Until 28 July
dropnaDataset.tail(1)

In [ ]:
# We will use this for visiualization
intubatedIntensive = dropnaDataset[["Date","Total Intensive Care","Total Intubated"]]
intubatedIntensive.set_index("Date",inplace=True)
intubatedIntensive.head()

In [ ]:
figint,(axint1,axint2) = plt.subplots(1,2,figsize=(12,9),tight_layout=True)
sns.barplot(y = intubatedIntensive.loc["28 March":"30 April"].index,x = intubatedIntensive.loc["28 March":"30 April"]["Total Intensive Care"].values,ax=axint1,color="purple",label="Total Intensive Care")
sns.barplot(y = intubatedIntensive.loc["28 March":"30 April"].index,x = intubatedIntensive.loc["28 March":"30 April"]["Total Intubated"].values,ax=axint1,color="orange",label="Total Intubated")
axint1.legend(loc="lower right",frameon=True)
axint1.grid(color="grey",alpha=0.3,linestyle="--",linewidth=1)
axint1.set_title("March & April",fontsize=20)
axint1.set_ylabel("Dates",fontsize=15)

sns.barplot(y = intubatedIntensive.loc["1 May":"31 May"].index,x = intubatedIntensive.loc["1 May":"31 May"]["Total Intensive Care"].values,ax=axint2,color="purple",label="Total Intensive Care")
sns.barplot(y = intubatedIntensive.loc["1 May":"31 May"].index,x = intubatedIntensive.loc["1 May":"31 May"]["Total Intubated"].values,ax=axint2,color="orange",label="Total Intubated")
axint2.legend(loc="lower right",frameon=True)
axint2.grid(color="grey",alpha=0.3,linestyle="--",linewidth=1)
axint2.set_title("May",fontsize=20)
axint2.set_ylabel("Dates",fontsize=15)

axint1.tick_params(width=3,color="black")
axint2.tick_params(width=3,color="black")
plt.show()

In [ ]:
figint2,(axint3,axint4) = plt.subplots(1,2,figsize=(12,9),tight_layout=True)
sns.barplot(y = intubatedIntensive.loc["1 June":"30 June"].index,x = intubatedIntensive.loc["1 June":"30 June"]["Total Intensive Care"].values,ax=axint3,color="purple",label="Total Intensive Care")
sns.barplot(y = intubatedIntensive.loc["1 June":"30 June"].index,x = intubatedIntensive.loc["1 June":"30 June"]["Total Intubated"].values,ax=axint3,color="orange",label="Total Intubated")
axint3.legend(loc="lower right",frameon=True)
axint3.grid(color="grey",alpha=0.3,linestyle="--",linewidth=1)
axint3.set_title("June",fontsize=20)
axint3.set_ylabel("Dates",fontsize=15)

sns.barplot(y = intubatedIntensive.loc["1 July":"28 July"].index,x = intubatedIntensive.loc["1 July":"28 July"]["Total Intensive Care"].values,ax=axint4,color="purple",label="Total Intensive Care")
sns.barplot(y = intubatedIntensive.loc["1 July":"28 July"].index,x = intubatedIntensive.loc["1 July":"28 July"]["Total Intubated"].values,ax=axint4,color="orange",label="Total Intubated")
axint4.legend(loc="lower right",frameon=True)
axint4.grid(color="grey",alpha=0.3,linestyle="--",linewidth=1)
axint4.set_title("July",fontsize=20)
axint4.set_ylabel("Dates",fontsize=15)

axint3.tick_params(width=3,color="black")
axint4.tick_params(width=3,color="black")
plt.show()

This is the change of intubated and intensive care patients. 

In [ ]:
# We will use this function at below
diff = dataset.drop(0,axis=0)["Total Intensive Care"].iloc[0]
def intchange(n):
    global diff
    res = n-diff
    diff = n
    return res

In [ ]:
# Added new column about intensive care change
dataset.insert(7,"Intensive Change",dataset.drop(0,axis=0)["Total Intensive Care"].apply(intchange))

In [ ]:
# We will use this function at below
difference = dataset.drop(0,axis=0)["Total Intubated"].iloc[0]
def intubatedchange(n):
    global difference
    res = n-difference
    difference = n
    return res

In [ ]:
# Added new column about intubated change
dataset.insert(9,"Intubated Change",dataset.drop(0,axis=0)["Total Intubated"].apply(intubatedchange))

In [ ]:
# Let's control this transactions
dataset.drop(0,axis=0).head()

**Analysis of intubated and intensive care changes**

In [ ]:
sns.jointplot(x = dataset["Intensive Change"],y = dataset["Intubated Change"],kind = "scatter",alpha=0.4,color="red",height=9,linewidth=10,edgecolor="red",ratio=4,facecolor="black")
plt.show()


Let's analyze other data that started to be **shared after July 28**.

<a id="11"></a>
# Numbers of Seriously Ill and Pneumonia Ratios

In [ ]:
seriouslyill.head()

<font style="font-size:16px;letter-spacing:0.5px;">The pneumonia rates you see here are expressed as a percentage.</font>

In [ ]:
firstpart = seriouslyill.iloc[:seriouslyill.Date.values.size//2]
secondpart= seriouslyill.iloc[seriouslyill.Date.values.size//2:]

In [ ]:
figser,(axser1,axser2) = plt.subplots(nrows=2,ncols=1,figsize=(12,9),tight_layout=True)
sns.barplot(x = firstpart.Date.values,y = firstpart.Seriously_ill.values,ax=axser1)
axser1.grid(color="grey",alpha=0.3,linestyle="--",linewidth=1)
axser1.set_title("Seriously Ill",fontsize=20)
axser1.tick_params(width=3,color="black")
axser1.set_yticks(np.arange(0,2500,200))
axser1.set_xticklabels(firstpart.Date.values,rotation=90)

sns.barplot(x = secondpart.Date.values,y = secondpart.Seriously_ill.values,ax=axser2)
axser2.grid(color="grey",alpha=0.3,linestyle="--",linewidth=1)
axser2.set_xlabel("Dates",fontsize=15)
axser2.tick_params(width=3,color="black")
axser2.set_yticks(np.arange(0,4400,300))
axser2.set_xticklabels(secondpart.Date.values,rotation=90)
plt.show()


<p style="font-size:18px;font-family:Arial;">As can be seen, <b>the increase</b> continues.</p>

In [ ]:
figpin,(axpin1,axpin2) = plt.subplots(nrows=2,ncols=1,figsize=(12,9),tight_layout=True)

axpin1.plot(firstpart.Date.values,firstpart.Pneumonia_ratio.values,lw=3,color="cyan",marker="o",markerfacecolor="blue",label="Pneumonia Ratio")
axpin1.set_yticks(np.arange(7,10,0.25))
axpin1.grid(color="grey",alpha=0.3,linestyle="--",linewidth=1)
axpin1.tick_params(width=3,color="black")
axpin1.set_title("Pneumonia Ratio",fontsize=20)
axpin1.set_ylabel("Ratio",fontsize=15)
axpin1.legend(loc="center right",frameon=True)
axpin1.set_xticklabels(firstpart.Date.values,rotation=90)

axpin2.plot(secondpart.Date.values,secondpart.Pneumonia_ratio.values,lw=3,color="cyan",marker="o",markerfacecolor="blue",label="Pneumonia Ratio")
axpin2.set_yticks(np.arange(3.5,7.25,0.25))
axpin2.grid(color="grey",alpha=0.3,linestyle="--",linewidth=1)
axpin2.tick_params(width=3,color="black")
axpin2.set_xlabel("Dates",fontsize=15)
axpin2.set_ylabel("Ratio",fontsize=15)
axpin2.legend(loc="center right",frameon=True)
axpin2.set_xticklabels(secondpart.Date.values,rotation=90)
plt.show()

<p style="font-size:18px;font-family:Arial;"> The decrease in the rate of pneumonia continues </p>

* <p style="font-size:16px;font-weight:700;">Thank you for your upvotes :)</p>